<a href="https://colab.research.google.com/github/NavSanya/MyRelationalDatabase/blob/main/NavSanya_6878418392.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
import re
import csv
import cmd
import argparse
import os
import math
import traceback
# os.chdir('/content/drive/Shareddrives/USC_DSCI551-  Foundation of Data Management/Project') # where the files for this project are

In [ ]:
# Function to copy selected columns from source CSV to a new CSV
def copyColumns(sourceCsvFile, destinationCsvFile, columnsToCopy):
    try:
        with open(sourceCsvFile, 'r', newline='', encoding='utf-8') as source_file:
            reader = csv.DictReader(source_file)
            source_headers = reader.fieldnames

            # Check if all columns to copy exist in the source CSV
            for column in columnsToCopy:
                if column not in source_headers:
                    print(f"Column '{column}' not found in the source CSV.")
                    return

            with open(destinationCsvFile, 'w', newline='', encoding='utf-8') as dest_file:
                writer = csv.DictWriter(dest_file, fieldnames=columnsToCopy)
                writer.writeheader()

                for row in reader:
                    # Extract the desired columns
                    selected_data = {col: str(row[col]) for col in columnsToCopy}
                    writer.writerow(selected_data)


        print("Selected columns copied to", destinationCsvFile)
        convert_csv_types(destinationCsvFile)

    except FileNotFoundError:
        print("Source CSV file not found.")
    except Exception as e:
        print("An error occurred:", str(e))

def convert_csv_types(fileName):
    try:
        with open(fileName, mode='r', newline='', encoding='utf-8') as input_csv_file:
            reader = csv.reader(input_csv_file)

            updatedData = []
            header = next(reader)  # Read the header row
            data_types = []  # List to store inferred data types for each column

            for column in header:
                if column.replace(".", "", 1).isdigit():
                    data_types.append(float)
                else:
                    data_types.append(str)

            for row in reader:
                converted_row = []
                for value, data_type in zip(row, data_types):
                    if data_type == float:
                        try:
                            converted_item = float(value)
                        except ValueError as e:
                            print(f"Value Error = {e}")
                            converted_item = None  # Keep it as a string for non-numeric values
                    else:
                        converted_item = value  # Keep it as a string
                    converted_row.append(converted_item)
                updatedData.append(converted_row)

        with open(fileName, mode='w', newline='', encoding='utf-8') as output_csv_file:
            writer = csv.writer(output_csv_file)
            writer.writerow(header)  # Write the header row
            writer.writerows(updatedData)

        # print(f"Data types converted and saved to {fileName}")

    except FileNotFoundError:
        print(f"File '{fileName}' not found.")
    except Exception as e:
        print("An error occurred:", str(e))


# loading all the tables
def createIndivisualTables():
    # Essentials Table
    copyColumns("IMDBTop250Movies.csv", "essentials.csv",["rank", "name", "year"])
    copyColumns("IMDBTop250Movies.csv", "viewerDeets.csv",["rank", "rating", "genre", "certificate", "run_time", "tagline"])
    copyColumns("IMDBTop250Movies.csv", "mediaDeets.csv",["rank", "budget", "tagline", "box_office"])
    copyColumns("IMDBTop250Movies.csv", "makers.csv",["rank", "casts", "directors", "writers"])


In [ ]:
# createIndivisualTables()

In [ ]:
class DatabaseFunctions:
        #actual db model functions

        def LoadCsvFile(self, fileName, chunkSize = 10):
            convert_csv_types(fileName)
            table = []
            with open(fileName, mode='r', newline='',  encoding='utf-8') as file:
                reader = csv.reader(file)
                header = next(reader, None)
                if header:
                    for row in reader:
                        data = {header[i]: value for i, value in enumerate(row)}
                        converted_dict = {}

                        for key, value in data.items():
                            try:
                                # Try to convert to int
                                converted_value = int(value)
                            except ValueError:
                                try:
                                    # If not an int, try to convert to float
                                    converted_value = float(value)
                                except ValueError:
                                    # If it can't be converted to a number, keep it as a string
                                    converted_value = value

                            converted_dict[key] = converted_value
                        table.append(converted_dict)

                        if len(table) >= chunkSize:
                            yield table # yeild the chunk of data
                            table = [] # reset table for the next chunk
                else:
                    table = list(reader)
            # print(*table, sep = '\n')
            yield table
            file.close()
            # return table

        #CREATE TABLE
        def createTable(self, table_name, columns):
            fileName = str(table_name) + ".csv"
            try:
                with open(fileName, mode='w', newline='', encoding='utf-8') as file:
                    writer = csv.writer(file)
                    writer.writerow(columns)
                    print(f"Table \'{table_name}\' created.")
            except FileNotFoundError:
                print(f"File '{fileName}' not found")
            except Exception as e:
                print("An error occurred:", str(e))

        #INSERT INTO
        def insertInto(self, table_name, columns, values):
            fileName = str(table_name) + ".csv"
            data = [dict(zip(columns, values))]
            try:
                with open(fileName, "a", newline="", encoding='utf-8') as f:
                    for row in data:
                        row_string = ",".join([row[column] for column in columns])
                        f.write(row_string + "\n")

                    convert_csv_types(fileName)
                    print(f"Data inserted into {table_name}")

            except FileNotFoundError:
                print(f"Table '{table_name}' not found. Creating the table...")
                # Create the table with the provided columns
                self.createTable(table_name, columns)
                # Retry the insertion
                self.insertInto(table_name, columns, values)

            except Exception as e:
                print("An error occurred:", str(e))
                traceback.print_exc()

        #GET FROM WHERE
        def get(self, table_name, columns, condition):
            fileName = f"{table_name}.csv"
            try:
                filtered_table = []
                i = 1
                for data in self.LoadCsvFile(fileName, 50):
                    if data == []:
                        break
                    print(f"Chunk {i} processing")
                    i+=1
                    header = list(data[0].keys())
                    if columns == [None]:
                        columns = header
                    for row in data:
                        if eval(condition, row):
                            filtered_table.append({column: row[column] for column in columns})
                    print(*filtered_table, sep='\n')
                return filtered_table

            except FileNotFoundError:
                print(f"Table '{table_name}' not found.")
            except Exception as e:
                print("An error occurred:", str(e))
                traceback.print_exc()

        # show table
        def displayTable(self, table_name):
            fileName = str(table_name) + ".csv"
            data = self.LoadCsvFile(fileName)
            data = data[:10]
            print(*data, sep="\n")

        # JOIN
        def joinTable(self, table1_name, table2_name, commonColumns):
            fileName1 = str(table1_name) + ".csv"
            fileName2 = str(table2_name) + ".csv"

            try:
                joinedData = []
                i = 1
                j = 1
                # Load table1 data in chunks
                for chunk1 in self.LoadCsvFile(fileName1, 50):
                    print(f"Chunk {i} of {table1_name} processing")
                    i+=1
                    # Load table2 data in chunks
                    for chunk2 in self.LoadCsvFile(fileName2, 50):
                        print(f"Chunk {j} of {table2_name} processing")
                        j+=1
                        # Perform join operation for each chunk pair
                        chunkJoinedData = []
                        # Join rows from table1 with matching rows from table2
                        for row1 in chunk1:
                            match = False
                            for row2 in chunk2:
                                match = True
                                for key in commonColumns:
                                    if row1.get(key) != row2.get(key):
                                        match = False
                                        break
                                if match:
                                    mergedRow = {**{f'{key}_{table1_name}': value for key, value in row1.items()},
                                                **{f'{key}_{table2_name}': row2.get(key) for key in row2}}
                                    chunkJoinedData.append(mergedRow)
                                    break

                        # Add unmatched rows from table1
                        for row1 in chunk1:
                            match = True
                            for row2 in chunk2:
                                for key in commonColumns:
                                    if row1.get(key) != row2.get(key):
                                        match = False
                                        break
                            if match:
                                mergedRow = {**{f'{key}_{table1_name}': value for key, value in row1.items()},
                                            **{f'{key}_{table2_name}': None for key in commonColumns}}
                                chunkJoinedData.append(mergedRow)

                        joinedData.extend(chunkJoinedData)

                print(*joinedData, sep='\n')

            except FileNotFoundError:
                print(f"Table(s) '{table1_name}' or '{table2_name}' not found.")
            except Exception as e:
                print("An error occurred:", str(e))
                import traceback
                traceback.print_exc()

        # AGGREGATE FUNCTION
        def aggregate(self, tablename, column, aggFunc, group_by=[]):
          fileName1 = tablename + ".csv"

          try:
              for chunk in self.LoadCsvFile(fileName1, 50):
                if group_by:
                    grouped_data = {}
                    for row in chunk:
                        key = tuple(row[col] for col in group_by)
                        if key not in grouped_data:
                            grouped_data[key] = []
                        grouped_data[key].append(row[column])

                    if aggFunc == "SUM":
                        result = {key: sum(values) for key, values in grouped_data.items()}
                    elif aggFunc == "AVG":
                        result = {key: sum(values) / len(values) for key, values in grouped_data.items()}
                    elif aggFunc == "COUNT":
                        result = {key: len(values) for key, values in grouped_data.items()}
                    elif aggFunc == "MAX":
                        result = {key: max(values) for key, values in grouped_data.items()}
                    elif aggFunc == "MIN":
                        result = {key: min(values) for key, values in grouped_data.items()}
                    else:
                        print(f"Unsupported aggregate function: {aggFunc}")
                        return None
                    # print(result)
                    yield result

                else:
                    actual_list = [row[column] for row in chunk]
                    if actual_list == []:
                        return
                    if aggFunc == "SUM":
                        result = sum(actual_list)
                    elif aggFunc == "AVG":
                        result = sum(actual_list) / len(actual_list)
                    elif aggFunc == "COUNT":
                        result = len(actual_list)
                    elif aggFunc == "MAX":
                        result = max(actual_list)
                    elif aggFunc == "MIN":
                        result = min(actual_list)
                    else:
                        print(f"Unsupported aggregate function: {aggFunc}")
                        return None
                    # print(result)
                    yield result

          except FileNotFoundError:
              print(f"Table '{tablename}' not found.")
          except Exception as e:
              print("An error occurred:", str(e))
              import traceback
              traceback.print_exc()



In [ ]:
class QueryLanguageProcessing:
    def __init__(self):
        self.inputQuery = ""
        self.variables = []

    def getCondition(self, strCondition):
        # Define mapping of words to symbols
        word_to_symbol = {
            'is greater than': '>',
            'is less than': '<',
            'is at least': '>=',
            'is at most': '<=',
            'is equal to': '==',
            'and': 'and',
            'or': 'or'
        }

        # Replace words with corresponding symbols
        for word, symbol in word_to_symbol.items():
            strCondition = strCondition.replace(word, symbol)

        # Replace 'x' and 'y' with their respective variable names
        strCondition = re.sub(r'\bx\b', 'x', strCondition)
        strCondition = re.sub(r'\by\b', 'y', strCondition)

        return strCondition

    def processStatement(self, statement, chunkSize = None):

        # regular expressions to match different parts of the statement
        getPattern = r"GET (.+) FROM TABLE (\w+) (?:WHERE (.+))?" #DONE
        combineTablesPattern = r"COMBINE TABLES (.+) ON (.+)" #DONE
        findAggrigateFuncPattern = r"FIND (\w+) OF COLUMN (.+) FROM TABLE (\w+)" # DONE
        AddValuesIntoPattern = r"ADD VALUES (.+) INTO TABLE (\w+) IN COLUMN/S (.+)" #DONE
        CreateTablePattern = r"CREATE A TABLE (.+) WITH THE FOLLOWING COLUMNS: (.+)" #DONE
        updatePattern = r"UPDATE (.+) IN TABLE (.+) WITH (.+)"
        showTable = r"SHOW TABLE (.+)" #DONE

        dbFuncObj = DatabaseFunctions()

        #get colums statement
        if re.match(getPattern, statement):
            match = re.match(getPattern, statement)
            columnNames = match.group(1).split(',')
            table = match.group(2)
            if match.group(3) is not None:
                whereConditionStr = match.group(3)
            else:
                whereConditionStr = "True"
            whereCondition = self.getCondition(whereConditionStr)
            print(f"Table Name = {table}, column name = {columnNames}, condition = {whereCondition}")
            # Process the SELECT statement here
            if columnNames == ['ALL']:
                result = dbFuncObj.get(table_name=table, columns=[None], condition=whereCondition)
            else:
                result = dbFuncObj.get(table_name=table, columns=columnNames, condition=whereCondition)

            print(*result, sep="\n")
            # print(result)

        #create table
        elif re.match(CreateTablePattern, statement):
            match = re.match(CreateTablePattern, statement)
            table = match.group(1)
            columns = match.group(2).split(",")
            # Process the CREATE TABLE statement here
            dbFuncObj.createTable(table,columns)

        #combineTable
        elif re.match(combineTablesPattern, statement):
            match = re.match(combineTablesPattern, statement)
            tableNames = match.group(1).split(',')
            columnNames = match.group(2).split(',')
            result = dbFuncObj.joinTable(tableNames[0], tableNames[1], columnNames)
            print(*result, sep="\n")

        #insert value
        elif re.match(AddValuesIntoPattern, statement):
            match = re.match(AddValuesIntoPattern, statement)
            values = match.group(1).split(",")
            tableName = match.group(2)
            columnNames = match.group(3).split(",")
            # Process the CREATE TABLE statement here
            print(f"Table name = {tableName} columns = {columnNames} values = {values}")
            dbFuncObj.insertInto(tableName, columnNames, values)

        #aggrigate function
        elif re.match(findAggrigateFuncPattern, statement):
          match = re.match(findAggrigateFuncPattern, statement)
          aggrigateFunc = match.group(1)
          column = match.group(2)
          tableName = match.group(3)
          #process aggrigate statement here
          print(f"Aggrigate func = {aggrigateFunc} column = {column} table name = {tableName}")
          convert_csv_types(tableName + ".csv")
          result = dbFuncObj.aggregate(tableName, column, aggrigateFunc)
          print(result)

        #display table
        elif re.match(showTable, statement):
            match = re.match(showTable, statement)
            tableName = match.group(1)
            # Process the CREATE TABLE statement here
            print(f"Table name = {tableName}")
            dbFuncObj.displayTable(tableName)



In [ ]:
# Create the CLI Interface
class MyDbCLI(cmd.Cmd):
        intro = "Welcome to MyDB! Type 'help' to list commands or 'EXIT' to quit."
        prompt = "MyDB> "
        # dbfunc = DatabaseFunctions()
        # queryObj = QueryLanguageProcessing()
        def __init__(self) :
            super().__init__()
            self.dbfunc = DatabaseFunctions()
            self.queryObj = QueryLanguageProcessing()

        def default(self, line):
          self.queryObj.processStatement(line)

        def do_GET(self, line):
          """
          It captures the columns to retrieve, the table name, and an optional WHERE condition for filtering the results.
          SYNTAX:  GET <columns> FROM TABLE <table> WHERE <condition>
                   GET ALL FROM TABLE <table> WHERE <condition>
          EXAMPLE: GET column1,column2 FROM TABLE myTable
                   GET ALL FROM TABLE employees WHERE department is equal to 'HR'

          WHERE CONDITION LIST:
                  greater than            | >
                  less than               | <
                  at least                | >=
                  at most                 | <=
                  equals                  | ==
                  and                     | and
                  or                      | or
          """
          self.queryObj.processStatement("GET " + str(line))

        def do_COMBINE(self, line):
          """
          This pattern is used to identify a command that combines two or more tables based on a specified column.
          SYNTAX:  COMBINE TABLES <tables> ON <column_name>
          EXAMPLE: COMBINE TABLES table1,table2 ON column
          """
          self.queryObj.processStatement("COMBINE " + str(line))

        def do_FIND(self, line):
          """
          This pattern is designed to identify a query that calculates an aggregate function (e.g., SUM, AVG, COUNT, MAX, MIN) on a specific column within a table.
          SYNTAX:  FIND <aggregate_function> OF COLUMN <column> FROM TABLE <table>
          EXAMPLE: FIND SUM OF COLUMN sales FROM TABLE sales_data
          """
          self.queryObj.processStatement("FIND " + str(line))

        def do_ADD(self, line):
          """
          This pattern captures a command for adding values into specified columns of a table.
          SYNTAX:  ADD VALUES <values> INTO TABLE <table> IN COLUMN/S <columns>
          EXAMPLE: ADD VALUES 'John',30 INTO TABLE employees IN COLUMNS 'name','age'
          """
          self.queryObj.processStatement("ADD" + str(line))

        def do_CREATE(self, line):
          """
          This pattern is used for creating a new table with a list of specified columns.
          SYNTAX:  CREATE A TABLE <table> WITH THE FOLLOWING COLUMNS: <columns>
          EXAMPLE: CREATE A TABLE products WITH THE FOLLOWING COLUMNS: product_id,product_name,price
          """
          self.queryObj.processStatement("CREATE" + str(line))

        def do_SHOW(self, line):
          """
          This pattern captures a command to display first 10 rows of a specific table
          SYNTAX:  SHOW TABLE <table>
          EXAMPLE: SHOW TABLE customers
          """
          print(line)
          self.queryObj.processStatement("SHOW " + str(line))

        def do_help(self, args):
          """
          Display help for available commands.
          Syntax: HELPME [<command_name>]
          Example: HELPME CREATE
          """
          if args:
              try:
                  command = getattr(self, f"do_{args}")
                  print(command.__doc__)
              except AttributeError:
                  print(f"Command '{args}' not found.")
          else:
              super().do_help(args)

        def do_exit(self, args):
          """
          Exit the MyDB CLI!
          """
          print("Exiting MyDB. Goodbye!")
          return True

# if __name__ == "__main__":
#   myCliObj = MyDbCLI()
#   myCliObj.cmdloop()



In [ ]:
#test dbfunctions

dbfunc = DatabaseFunctions()
# # list = islice(dbfunc.LoadCsvFile("makers.csv"), 10)
# # print(*list, sep="\n")

# #create table - DONE
# dbfunc.createTable(table_name="testTable", columns=["a","b","c"])

# # insert into - DONE
# dbfunc.insertInto("testTable", ["a","b", "c"], ["2", "1.2", "yo"])

# # display Table - DONE
# dbfunc.displayTable("makers")

# # get function - DONE
# res = dbfunc.get("makers", ["rank", "directors"], "rank >= 90 and rank < 100")
# print(res)
# print(*res, sep="\n")

# # join - DONE
# result = dbfunc.joinTable("testTable", "testTableQuery", ["a", "b"])

# aggregate - DONE
result = dbfunc.aggregate("makers", "rank", "AVG")
res = list(result)
print(sum(res)/len(res))



125.5


In [ ]:

# Test Statement processing of queries

queryObj = QueryLanguageProcessing()

# #getCondition
# x = 10
# y = 20
# english_sentence = "True"
# conditional_statement = queryObj.getCondition(english_sentence)
# print("English Sentence:", english_sentence)
# print("Conditional Statement:", conditional_statement)
# print(eval(conditional_statement))

# #get statement
# statement = "GET rank,writers FROM TABLE makers WHERE rank is greater than 0 and rank is less than 26"
# queryObj.processStatement(statement, chunkSize=50)
# # print(*result, sep="\n")

# # create Table
# statement = "CREATE A TABLE testTableQuery WITH THE FOLLOWING COLUMNS: a,b"
# result = queryObj.processStatement(statement)
# # print(*result, sep="\n")

# #get all statement
# statement = "GET ALL FROM TABLE mediaDeets WHERE budget is at least box_office"
# result = queryObj.processStatement(statement)
# print(*result, sep="\n")

# # AddValues
# statement = "ADD VALUES 1,2.1 INTO TABLE testTableQuery IN COLUMN/S a,b"
# result = queryObj.processStatement(statement)
# print(result)

# # ShowTable
# statement = "SHOW TABLE makers"
# queryObj.processStatement(statement)

# # combineTable
# statement = "COMBINE TABLES testTable,testTableQuery ON a"
# queryObj.processStatement(statement)

# # aggregate
# statement = "FIND COUNT OF COLUMN a FROM TABLE testTable"
# queryObj.processStatement(statement)

In [ ]:
#EXTRAA CODE
# def readCsvInChunks(self, filename, chunk_size=10):
        #     with open(filename, 'r', encoding='utf-8') as csvfile:
        #         # Read and store the header
        #         header = csvfile.readline().strip().split(',')

        #         # Initialize an empty set to track processed chunks
        #         processed_chunks = set()

        #         while True:
        #             chunk = []
        #             for _ in range(chunk_size):
        #                 row = csvfile.readline().strip().split(',')
        #                 if not row:
        #                     break  # Break if end of file is reached

        #                 chunk.append(row)

        #             if chunk:
        #                 # Check if the current chunk has already been processed
        #                 chunk_hash = hash(tuple(tuple(row) for row in chunk))
        #                 if chunk_hash in processed_chunks:
        #                     # Skip processing the chunk if it's a repeat
        #                     continue

        #                 # Add the current chunk to the processed_chunks set
        #                 processed_chunks.add(chunk_hash)

        #                 yield header, chunk
        #             else:
        #                 break

        # def LoadCsvFile(self, filename, chunk_size=10):
        #     convert_csv_types(filename)
        #     table = []

        #     with open(filename, mode='r', newline='', encoding='utf-8') as file:
        #         header, _ = file.readline().strip().split(','), None
        #         print("########")

        #         if header:
        #             for chunk in self.readCsvInChunks(file, chunk_size):
        #                 for row in chunk:
        #                     data = {header[i]: value for i, value in enumerate(row)}
        #                     converted_dict = {}

        #                     for key, value in data.items():
        #                         try:
        #                             # Try to convert to int
        #                             converted_value = int(value)
        #                         except ValueError:
        #                             try:
        #                                 # If not an int, try to convert to float
        #                                 converted_value = float(value)
        #                             except ValueError:
        #                                 # If it can't be converted to a number, keep it as a string
        #                                 converted_value = value

        #                         converted_dict[key] = converted_value
        #                     table.append(converted_dict)
        #         else:
        #             table = [line.strip().split(',') for line in file]

        #     print(table)
        #     return table





#################### BEFORE CHUNKS #############################################################
# from google.colab import drive
# drive.mount('/content/drive')

# import re
# import csv
# import cmd
# import argparse
# import os
# import math
# os.chdir('/content/drive/Shareddrives/USC_DSCI551-  Foundation of Data Management/Project') # where the files for this project are

# # Function to copy selected columns from source CSV to a new CSV
# def copyColumns(sourceCsvFile, destinationCsvFile, columnsToCopy):
#     try:
#         with open(sourceCsvFile, 'r', newline='', encoding='utf-8') as source_file:
#             reader = csv.DictReader(source_file)
#             source_headers = reader.fieldnames

#             # Check if all columns to copy exist in the source CSV
#             for column in columnsToCopy:
#                 if column not in source_headers:
#                     print(f"Column '{column}' not found in the source CSV.")
#                     return

#             with open(destinationCsvFile, 'w', newline='', encoding='utf-8') as dest_file:
#                 writer = csv.DictWriter(dest_file, fieldnames=columnsToCopy)
#                 writer.writeheader()

#                 for row in reader:
#                     # Extract the desired columns
#                     selected_data = {col: str(row[col]) for col in columnsToCopy}
#                     writer.writerow(selected_data)


#         print("Selected columns copied to", destinationCsvFile)
#         convert_csv_types(destinationCsvFile)

#     except FileNotFoundError:
#         print("Source CSV file not found.")
#     except Exception as e:
#         print("An error occurred:", str(e))

# def convert_csv_types(fileName):
#     try:
#         with open(fileName, mode='r', newline='', encoding='utf-8') as input_csv_file:
#             reader = csv.reader(input_csv_file)

#             updatedData = []
#             header = next(reader)  # Read the header row
#             data_types = []  # List to store inferred data types for each column

#             for column in header:
#                 if column.replace(".", "", 1).isdigit():
#                     data_types.append(float)
#                 else:
#                     data_types.append(str)

#             for row in reader:
#                 converted_row = []
#                 for value, data_type in zip(row, data_types):
#                     if data_type == float:
#                         try:
#                             converted_item = float(value)
#                         except ValueError as e:
#                             print(f"Value Error = {e}")
#                             converted_item = None  # Keep it as a string for non-numeric values
#                     else:
#                         converted_item = value  # Keep it as a string
#                     converted_row.append(converted_item)
#                 updatedData.append(converted_row)

#         with open(fileName, mode='w', newline='', encoding='utf-8') as output_csv_file:
#             writer = csv.writer(output_csv_file)
#             writer.writerow(header)  # Write the header row
#             writer.writerows(updatedData)

#         # print(f"Data types converted and saved to {fileName}")

#     except FileNotFoundError:
#         print(f"File '{fileName}' not found.")
#     except Exception as e:
#         print("An error occurred:", str(e))


# # # loading all the tables
# # def createIndivisualTables():
# #     # Essentials Table
# #     copyColumns("IMDBTop250Movies.csv", "essentials.csv",["rank", "name", "year"])
# #     copyColumns("IMDBTop250Movies.csv", "viewerDeets.csv",["rank", "rating", "genre", "certificate", "run_time", "tagline"])
# #     copyColumns("IMDBTop250Movies.csv", "mediaDeets.csv",["rank", "budget", "tagline", "box_office"])
# #     copyColumns("IMDBTop250Movies.csv", "makers.csv",["rank", "casts", "directors", "writers"])


# # createIndivisualTables()

# class ChunkProcessor:
#     def __init__(self, chunk_size=50):
#         self.chunk_size = chunk_size

#     def process_large_dataset(self, source_csv_file, query_columns):
#         try:
#             # Get the total number of rows in the dataset
#             total_rows = sum(1 for line in open(source_csv_file, encoding='utf-8'))

#             # Calculate the number of chunks based on the chunk size
#             num_chunks = (total_rows // self.chunk_size) + 1

#             with open(source_csv_file, 'r', newline='', encoding='utf-8') as csvfile:
#                 # Initialize a CSV reader with chunk size
#                 reader = csv.DictReader(csvfile)
#                 fieldnames = reader.fieldnames

#                 for i in range(num_chunks):
#                     # Read a chunk of data
#                     chunk = list(self.read_chunk(reader, fieldnames))

#                     # Process each chunk independently
#                     result_chunk = self.process_chunk(chunk, query_columns)

#                     # Display or save the results of each chunk
#                     self.display_result(result_chunk)

#                     # Optional: Save the processed chunk to a new CSV file
#                     self.save_result_to_csv(result_chunk, f'result_chunk_{i}.csv', fieldnames)

#         except FileNotFoundError:
#             print(f"File '{source_csv_file}' not found.")
#         except Exception as e:
#             print("An error occurred:", str(e))

#     def read_chunk(self, reader, fieldnames):
#         for _ in range(self.chunk_size):
#             try:
#                 yield next(reader)
#             except StopIteration:
#                 break

#     def process_chunk(self, chunk, query_columns):
#         # Implement your query or processing logic here for each chunk
#         # For demonstration, let's select specified columns from the chunk
#         result_chunk = [{col: row[col] for col in query_columns} for row in chunk]
#         return result_chunk

#     def display_result(self, result_chunk):
#         # Implement how you want to display the result for each chunk
#         print(result_chunk)

#     def save_result_to_csv(self, result_chunk, output_filename, fieldnames):
#         # Implement how you want to save the result to a new CSV file for each chunk
#         with open(output_filename, 'w', newline='', encoding='utf-8') as csvfile:
#             writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
#             writer.writeheader()
#             writer.writerows(result_chunk)
#         print(f"Result saved to {output_filename}")


# class DatabaseFunctions:
#         #actual db model functions
#         def LoadCsvFile(self, fileName):
#             convert_csv_types(fileName)
#             table = []
#             with open(fileName, mode='r', newline='',  encoding='utf-8') as file:
#                 reader = csv.reader(file)
#                 header = next(reader, None)
#                 if header:
#                     for row in reader:
#                         data = {header[i]: value for i, value in enumerate(row)}
#                         converted_dict = {}

#                         for key, value in data.items():
#                             try:
#                                 # Try to convert to int
#                                 converted_value = int(value)
#                             except ValueError:
#                                 try:
#                                     # If not an int, try to convert to float
#                                     converted_value = float(value)
#                                 except ValueError:
#                                     # If it can't be converted to a number, keep it as a string
#                                     converted_value = value

#                             converted_dict[key] = converted_value
#                         table.append(converted_dict)
#                 else:
#                     table = list(reader)
#             print(table)
#             file.close()
#             return table

#         #CREATE TABLE
#         def createTable(self, table_name, columns):
#             fileName = str(table_name) + ".csv"
#             try:
#                 with open(fileName, mode='w', newline='', encoding='utf-8') as file:
#                     writer = csv.writer(file)
#                     writer.writerow(columns)
#                     print(f"Table \'{table_name}\' created.")
#             except FileNotFoundError:
#                 print(f"File '{fileName}' not found")
#             except Exception as e:
#                 print("An error occurred:", str(e))

#         #INSERT INTO
#         def insertInto(self, table_name, columns, values):
#             fileName = str(table_name) + ".csv"
#             try:
#                 with open(fileName, mode='r', newline='', encoding='utf-8') as fileRead:
#                     reader = csv.reader(fileRead)
#                     currentData = list(reader)

#                 # Find the column indices to insert data into
#                 column_indices = [currentData[0].index(col) for col in columns]

#                 # Create a new row with empty values for all columns
#                 new_row = [''] * len(currentData[0])

#                 # Populate the new row with the provided values in the specified columns
#                 for col, val in zip(column_indices, values):
#                     new_row[col] = val

#                 # Append the new row to the existing data
#                 currentData.append(new_row)

#                 # Write the updated data back to the CSV file
#                 with open(fileName, mode='w', newline='', encoding='utf-8') as fileWrite:
#                     writer = csv.writer(fileWrite)
#                     writer.writerows(currentData)
#                 fileRead.close()
#                 fileWrite.close()
#                 convert_csv_types(fileName)
#                 print(f"Data inserted into {table_name}")

#             except FileNotFoundError:
#                 print(f"Table '{table_name}' not found. Creating the table...")
#                 # Create the table with the provided columns
#                 self.createTable(table_name, columns)
#                 # Retry the insertion
#                 self.insertInto(table_name, columns, values)

#             except Exception as e:
#                 print("An error occurred:", str(e))


#         #GET FROM WHERE
#         def get(self, table_name, columns, condition):
#             fileName = f"{table_name}.csv"
#             try:
#                 selectedData = []
#                 with open(fileName, mode='r', newline='', encoding='utf-8') as input_csv_file:
#                     reader = csv.reader(input_csv_file)
#                     header = next(reader)  # Read the header row
#                     data_types = []  # List to store inferred data types for each column
#                     row1 = next(reader)
#                     for column in row1:
#                         if column.isdigit():
#                             data_types.append(int)
#                         elif column.replace(".", "", 1).isdigit():
#                             data_types.append(float)
#                         else:
#                             data_types.append(str)

#                     # Read and convert data rows based on inferred data types
#                     for row in reader:
#                         converted_row = []
#                         for value, data_type in zip(row, data_types):
#                             if data_type == int:
#                                 try:
#                                     converted_row.append(int(value,20))
#                                 except ValueError as e:
#                                     converted_row.append(-1)  # Keep it as a string for non-numeric values

#                             elif data_type == float:
#                                 try:
#                                     converted_row.append(float(value))
#                                 except ValueError as e:
#                                     converted_row.append(-1.0)

#                             else:
#                                 converted_row.append(value)

#                         if columns == [None]:
#                             columns = header
#                         # Evaluate the condition and select rows that match
#                         if condition:
#                             if eval(condition, {}, dict(zip(header, converted_row))):
#                                 selected_row = {column: value for column, value in zip(header, converted_row) if column in columns}
#                                 selectedData.append(selected_row)
#                 return selectedData

#             except FileNotFoundError:
#                 print(f"Table '{table_name}' not found.")
#             except Exception as e:
#                 print("An error occurred:", str(e))

#         # show table
#         def displayTable(self, table_name):
#             fileName = str(table_name) + ".csv"
#             data = self.LoadCsvFile(fileName)
#             data = data[:10]
#             print(*data, sep="\n")

#         # JOIN
#         def joinTable(self, table1_name, table2_name, commonColumns):
#             fileName1 = str(table1_name) + ".csv"
#             fileName2 = str(table2_name) + ".csv"

#             try:
#                 tableData1 = self.LoadCsvFile(fileName1)
#                 tableData2 = self.LoadCsvFile(fileName2)
#                 # Perform the join operation
#                 joinedData = []
#                 for row1 in tableData1:
#                   for row2 in tableData2:
#                     match = True
#                     for key in commonColumns:
#                       if row1.get(key) != row2.get(key):
#                         match = False
#                         break
#                     if match:
#                       mergedRow = {**{f'{key}_{table1_name}': value for key, value in row1.items()},
#                         **{f'{key}_{table2_name}': row2.get(key) for key in row2}}
#                       joinedData.append(mergedRow)

#                 return joinedData

#             except FileNotFoundError:
#                 print(f"Table(s) '{table1_name}' or '{table2_name}' not found.")
#             except Exception as e:
#                 print("An error occurred:", str(e))
#                 import traceback
#                 traceback.print_exc()

#         # AGGREGATE FUNCTION
#         def aggregate(self, tablename, column, aggFunc, group_by=[]):
#           fileName1 = tablename + ".csv"

#           try:
#               tableData = self.LoadCsvFile(fileName1)
#               # print(tableData)
#               if group_by:
#                   print("Yes Goup by")
#                   grouped_data = {}
#                   for row in tableData:
#                       key = tuple(row[col] for col in group_by)
#                       if key not in grouped_data:
#                           grouped_data[key] = []
#                       grouped_data[key].append(row[column])

#                   if aggFunc == "SUM":
#                       result = {key: sum(values) for key, values in grouped_data.items()}
#                   elif aggFunc == "AVG":
#                       result = {key: sum(values) / len(values) for key, values in grouped_data.items()}
#                   elif aggFunc == "COUNT":
#                       result = {key: len(values) for key, values in grouped_data.items()}
#                   elif aggFunc == "MAX":
#                       result = {key: max(values) for key, values in grouped_data.items()}
#                   elif aggFunc == "MIN":
#                       result = {key: min(values) for key, values in grouped_data.items()}
#                   else:
#                       print(f"Unsupported aggregate function: {aggFunc}")
#                       return None

#                   return result

#               else:
#                   actual_list = [row[column] for row in tableData]
#                   if aggFunc == "SUM":
#                     result = sum(actual_list)
#                   elif aggFunc == "AVG":
#                       result = sum(actual_list) / len(actual_list)
#                   elif aggFunc == "COUNT":
#                       result = len(actual_list)
#                   elif aggFunc == "MAX":
#                       result = max(actual_list)
#                   elif aggFunc == "MIN":
#                       result = min(actual_list)
#                   else:
#                       print(f"Unsupported aggregate function: {aggFunc}")
#                       return None

#                   return result

#           except FileNotFoundError:
#               print(f"Table '{tablename}' not found.")
#           except Exception as e:
#               print("An error occurred:", str(e))
#               import traceback
#               traceback.print_exc()



# class QueryLanguageProcessing:
#     def __init__(self):
#         self.inputQuery = ""
#         self.variables = []

#     def getCondition(self, strCondition):
#         # Define mapping of words to symbols
#         word_to_symbol = {
#             'is greater than': '>',
#             'is less than': '<',
#             'is at least': '>=',
#             'is greater than or equal to': '>=',
#             'is at most': '<=',
#             'is less than or equal to': '>=',
#             'is equal to': '==',
#             'and': 'and',
#             'or': 'or'
#         }

#         # Replace words with corresponding symbols
#         for word, symbol in word_to_symbol.items():
#             strCondition = strCondition.replace(word, symbol)

#         # Replace 'x' and 'y' with their respective variable names
#         strCondition = re.sub(r'\bx\b', 'x', strCondition)
#         strCondition = re.sub(r'\by\b', 'y', strCondition)

#         return strCondition

#     def processStatement(self, statement):

#         # regular expressions to match different parts of the statement
#         getPattern = r"GET (.+) FROM TABLE (\w+) (?:WHERE (.+))?" #DONE
#         combineTablesPattern = r"COMBINE TABLES (.+) ON (.+)" #DONE
#         findAggrigateFuncPattern = r"FIND (\w+) OF COLUMN (.+) FROM TABLE (\w+)" # DONE
#         AddValuesIntoPattern = r"ADD VALUES (.+) INTO TABLE (\w+) IN COLUMN/S (.+)" #DONE
#         CreateTablePattern = r"CREATE A TABLE (.+) WITH THE FOLLOWING COLUMNS: (.+)" #DONE
#         updatePattern = r"UPDATE (.+) IN TABLE (.+) WITH (.+)"
#         showTable = r"SHOW TABLE (.+)" #DONE

#         dbFuncObj = DatabaseFunctions()

#         #get colums statement
#         if re.match(getPattern, statement):
#             match = re.match(getPattern, statement)
#             columnNames = match.group(1).split(',')
#             table = match.group(2)
#             if match.group(3) is not None:
#                 whereConditionStr = match.group(3)
#             else:
#                 whereConditionStr = "True"
#             whereCondition = self.getCondition(whereConditionStr)
#             print(f"Table Name = {table}, column name = {columnNames}, condition = {whereCondition}")
#             # Process the SELECT statement here
#             if columnNames == ['ALL']:
#               result = dbFuncObj.get(table_name=table, columns=[None], condition=whereCondition)
#             else:
#               result = dbFuncObj.get(table_name=table, columns=columnNames, condition=whereCondition)
#             print(*result, sep="\n")

#         #create table
#         elif re.match(CreateTablePattern, statement):
#             match = re.match(CreateTablePattern, statement)
#             table = match.group(1)
#             columns = match.group(2).split(",")
#             # Process the CREATE TABLE statement here
#             dbFuncObj.createTable(table,columns)

#         #combineTable
#         elif re.match(combineTablesPattern, statement):
#             match = re.match(combineTablesPattern, statement)
#             tableNames = match.group(1).split(',')
#             columnNames = match.group(2).split(',')
#             result = dbFuncObj.joinTable(tableNames[0], tableNames[1], columnNames)
#             print(*result, sep="\n")

#         #insert value
#         elif re.match(AddValuesIntoPattern, statement):
#             match = re.match(AddValuesIntoPattern, statement)
#             values = match.group(1).split(",")
#             tableName = match.group(2)
#             columnNames = match.group(3).split(",")
#             # Process the CREATE TABLE statement here
#             print(f"Table name = {tableName} columns = {columnNames} values = {values}")
#             dbFuncObj.insertInto(tableName, columnNames, values)

#         #aggrigate function
#         elif re.match(findAggrigateFuncPattern, statement):
#           match = re.match(findAggrigateFuncPattern, statement)
#           aggrigateFunc = match.group(1)
#           column = match.group(2)
#           tableName = match.group(3)
#           #process aggrigate statement here
#           print(f"Aggrigate func = {aggrigateFunc} column = {column} table name = {tableName}")
#           convert_csv_types(tableName + ".csv")
#           result = dbFuncObj.aggregate(tableName, column, aggrigateFunc)
#           print(result)

#         #display table
#         elif re.match(showTable, statement):
#             match = re.match(showTable, statement)
#             tableName = match.group(1)
#             # Process the CREATE TABLE statement here
#             print(f"Table name = {tableName}")
#             dbFuncObj.displayTable(tableName)

#         return "Done"

# # Create the CLI Interface
# class MyDbCLI(cmd.Cmd):
#         intro = "Welcome to MyDB! Type 'help' to list commands or 'EXIT' to quit."
#         prompt = "MyDB> "
#         # dbfunc = DatabaseFunctions()
#         # queryObj = QueryLanguageProcessing()
#         def __init__(self) :
#             super().__init__()
#             self.dbfunc = DatabaseFunctions()
#             self.queryObj = QueryLanguageProcessing()

#         def default(self, line):
#           res = self.queryObj.processStatement(line)
#           print(res)

#         def do_GET(self, line):
#           """
#           It captures the columns to retrieve, the table name, and an optional WHERE condition for filtering the results.
#           SYNTAX:  GET <columns> FROM TABLE <table> WHERE <condition>
#                    GET ALL FROM TABLE <table> WHERE <condition>
#           EXAMPLE: GET column1,column2 FROM TABLE myTable
#                    GET ALL FROM TABLE employees WHERE department is equal to 'HR'

#           WHERE CONDITION LIST:
#                   greater than            | >
#                   less than               | <
#                   at least                | >=
#                   greater than equal to   | >=
#                   greater than or equal to| >=
#                   at most                 | <=
#                   less than equal to      | <=
#                   less than or equal to   | <=
#                   equals                  | ==
#                   equal to                | ==
#           """
#           res = self.queryObj.processStatement("GET " + str(line))
#           print(res)

#         def do_COMBINE(self, line):
#           """
#           This pattern is used to identify a command that combines two or more tables based on a specified column.
#           SYNTAX:  COMBINE TABLES <tables> ON <column_name>
#           EXAMPLE: COMBINE TABLES table1,table2 ON column
#           """
#           res = self.queryObj.processStatement("COMBINE " + str(line))
#           print(res)

#         def do_FIND(self, line):
#           """
#           This pattern is designed to identify a query that calculates an aggregate function (e.g., SUM, AVG, COUNT, MAX, MIN) on a specific column within a table.
#           SYNTAX:  FIND <aggregate_function> OF COLUMN <column> FROM TABLE <table>
#           EXAMPLE: FIND SUM OF COLUMN sales FROM TABLE sales_data
#           """
#           res = self.queryObj.processStatement("FIND " + str(line))
#           print(res)

#         def do_ADD(self, line):
#           """
#           This pattern captures a command for adding values into specified columns of a table.
#           SYNTAX:  ADD VALUES <values> INTO TABLE <table> IN COLUMN/S <columns>
#           EXAMPLE: ADD VALUES 'John',30 INTO TABLE employees IN COLUMNS 'name','age'
#           """
#           res = self.queryObj.processStatement("ADD" + str(line))
#           print(res)

#         def do_CREATE(self, line):
#           """
#           This pattern is used for creating a new table with a list of specified columns.
#           SYNTAX:  CREATE A TABLE <table> WITH THE FOLLOWING COLUMNS: <columns>
#           EXAMPLE: CREATE A TABLE products WITH THE FOLLOWING COLUMNS: product_id,product_name,price
#           """
#           res = self.queryObj.processStatement("CREATE" + str(line))
#           print(res)

#         def do_SHOW(self, line):
#           """
#           This pattern captures a command to display first 10 rows of a specific table
#           SYNTAX:  SHOW TABLE <table>
#           EXAMPLE: SHOW TABLE customers
#           """
#           print(line)
#           res = self.queryObj.processStatement("SHOW " + str(line))
#           print(res)

#         def do_help(self, args):
#           """
#           Display help for available commands.
#           Syntax: HELPME [<command_name>]
#           Example: HELPME CREATE
#           """
#           if args:
#               try:
#                   command = getattr(self, f"do_{args}")
#                   print(command.__doc__)
#               except AttributeError:
#                   print(f"Command '{args}' not found.")
#           else:
#               super().do_help(args)

#         def do_exit(self, args):
#           """
#           Exit the MyDB CLI!
#           """
#           print("Exiting MyDB. Goodbye!")
#           return True

# if __name__ == "__main__":
#   myCliObj = MyDbCLI()
#   myCliObj.cmdloop()



# #test dbfunctions

# dbfunc = DatabaseFunctions()
# # # list = islice(dbfunc.LoadCsvFile("makers.csv"), 10)
# # # print(*list, sep="\n")

# # #create table - DONE
# # dbfunc.createTable(table_name="testTable", columns=["a","b","c"])

# # # insert into - DONE
# # dbfunc.insertInto("testTable", ["a","b", "c"], ["2", "1.2", "yo"])

# # # display Table - DONE
# # dbfunc.displayTable("makers")

# # # get function - DONE
# # res = dbfunc.get("makers", ["rank", "directors"], "rank >= 90 and rank < 100")
# # print(res)
# # # print(*res, sep="\n")

# # # join - DONE
# # result = dbfunc.joinTable("testTable", "testTableQuery", ["a"])
# # print(*result, sep="\n")

# # # aggregate - DONE
# # result = dbfunc.aggregate("testTable", "a", "AVG")
# # print(result)



# # Test Statement processing of queries

# queryObj = QueryLanguageProcessing()

# # #getCondition
# # x = 10
# # y = 20
# # english_sentence = "True"
# # conditional_statement = queryObj.getCondition(english_sentence)
# # print("English Sentence:", english_sentence)
# # print("Conditional Statement:", conditional_statement)
# # print(eval(conditional_statement))

# # #get statement
# # statement = "GET rank,writers FROM TABLE makers WHERE rank is greater than 20 and rank is less than 25"
# # result = queryObj.processStatement(statement)
# # print(*result, sep="\n")

# # # create Table
# # statement = "CREATE A TABLE testTableQuery WITH THE FOLLOWING COLUMNS: a,b"
# # result = queryObj.processStatement(statement)
# # # print(*result, sep="\n")

# # #get all statement
# # statement = "GET ALL FROM TABLE mediaDeets WHERE budget is at least box_office"
# # result = queryObj.processStatement(statement)
# # print(*result, sep="\n")

# # # AddValues
# # statement = "ADD VALUES 1,2.1 INTO TABLE testTableQuery IN COLUMN/S a,b"
# # result = queryObj.processStatement(statement)
# # print(result)

# # # ShowTable
# # statement = "SHOW TABLE makers"
# # queryObj.processStatement(statement)

# # # combineTable
# # statement = "COMBINE TABLES testTable,testTableQuery ON a"
# # queryObj.processStatement(statement)

# # # aggregate
# # statement = "FIND COUNT OF COLUMN a FROM TABLE testTable"
# # queryObj.processStatement(statement)
